# Introducing the Dataset

In [1]:
# Some initialization magic
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import collections

# Let's load up the data
import json
import gzip
import codecs

with gzip.GzipFile("scrapyproject/results.json.gz") as fin:
    fin = codecs.getreader('utf-8')(fin)
    tables = [json.loads(line) for line in fin]

In [2]:
# Here's what it looks like. 
# Each line corresponds to a table scraped from the Central Electoral Committee website.
# Tables are dictionaries.
sorted(tables[0].keys())

['column_headers',
 'data',
 'data_type',
 'md5',
 'region',
 'row_headers',
 'timestamp',
 'url']

In [3]:
#
# There are four kinds of data.
#     results_tik: Summary results for a territory IK (electoral committee)
#     results_uik: Detailed results for a territory IK, showing individual polling stations
#     turnout_tik: Sumarry of turnout for a territory IK
#     turnout_uik: Detailed turnout for a territory IK, showing individual polling stations
#
import collections
collections.Counter([t['data_type'] for t in tables])

Counter({'results_tik': 87,
         'results_uik': 2776,
         'turnout_tik': 87,
         'turnout_uik': 2776})

In [4]:
# region contains the name of the region.
a_table = [t for t in tables if t["data_type"] == "results_tik"][0]
a_table["region"]

'Республика Бурятия'

In [5]:
# url corresponds to the URL from which the data was scraped from
a_table["url"]

'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1000004&tvd=100100084849070&vrn=100100084849062&region=0&global=true&sub_region=0&prver=0&pronetvd=null&vibid=100100084849070&type=227'

In [6]:
# the timestamp shows the date and time that the data was scraped
a_table["timestamp"]

'2018-03-19T08:00:11.766832+00:00'

In [7]:
# md5 corresponds to the MD5 hash of the HTML that was fetched from the URL at the time of scraping
import requests
import hashlib
r = requests.get(a_table["url"])
assert hashlib.md5(r.content).hexdigest() == a_table["md5"], "MD5 mismatch, perhaps data is out of date?"

In [8]:
# Each table, regardless of data_type, contains data in rows and columns.
# The label for each row and column is contained in "row_headers" and "column_headers", respectively.
# For turnout tables, the data looks like this:
# The row headers in this case are the names of the local electoral committees (участковая избирательная коммиссия).
# The column headers are the times at which turnout was reported.
turnout_table = [t for t in tables if t["data_type"] == "turnout_tik"][0]
print(turnout_table["row_headers"])
print(turnout_table["column_headers"])

['ВСЕГО, в том числе', 'Джейрахская', 'Карабулакская городская', 'Магасская городская', 'Малгобекская городская', 'Малгобекская', 'Назрановская городская', 'Назрановская', 'Сунженская', 'Сунженская городская']
['10:00', '12:00', '15:00', '18:00']


In [9]:
# For result tables, the data looks like this:
results_table = [t for t in tables if t["data_type"] == "results_uik"][0]
print(results_table["row_headers"][:5])  # snipped for brevity
print(results_table["column_headers"][:5]) # snipped for brevity

['Число избирателей, включенных в список избирателей', 'Число избирательных бюллетеней, полученных участковой избирательной комиссией', 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно', 'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования', 'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования']
['Сумма', 'УИК №871', 'УИК №872', 'УИК №873', 'УИК №874']


In [10]:
ROW_HEADERS = results_table['row_headers']
ROW_HEADERS

['Число избирателей, включенных в список избирателей',
 'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
 'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
 'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования',
 'Число погашенных избирательных бюллетеней',
 'Число избирательных бюллетеней в переносных ящиках для голосования',
 'Число бюллетеней в стационарных ящиках для голосования',
 'Число недействительных избирательных бюллетеней',
 'Число действительных избирательных бюллетеней',
 'Число утраченных избирательных бюллетеней',
 'Число избирательных бюллетеней, не учтенных при получении',
 'Бабурин Сергей Николаевич',
 'Грудинин Павел Николаевич',
 'Жириновский Владимир Вольфович',
 'Путин Владимир Владимирович',
 'Собчак Ксения Анатольевна',
 'Сурайкин Максим Александрович',
 'Титов Борис Юрь

In [11]:
EN_ROW_HEADERS = (
    'Number of eligible voters',
    'Number of voting forms received by the polling station',
    'Number of voting forms distributed to early voters',
    'Number of voting forms distributed at the polling station on the day of the election',
    'Number of voting forms distributed outside the polling station on the day of the election',
    'Number of destroyed voting forms',
    'Number of voting forms in mobile boxes',
    'Number of voting forms in stationary boxes',
    'Number of invalid voting forms',
    'Number of valid voting forms',
    'Number of lost voting forms',
    'Number of voting forms that were received but not counted',
    'Sergey Baburin',
    'Pavel Grudinin',
    'Vladimir Zhirinovsky',
    'Vladimir Putin',
    'Xenia Sobchak',
    'Maxim Suraykin',
    'Boris Titov',
    'Grigory Yavlinsky'
)
assert len(EN_ROW_HEADERS) == len(ROW_HEADERS)
list(zip(ROW_HEADERS, EN_ROW_HEADERS))

[('Число избирателей, включенных в список избирателей',
  'Number of eligible voters'),
 ('Число избирательных бюллетеней, полученных участковой избирательной комиссией',
  'Number of voting forms received by the polling station'),
 ('Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
  'Number of voting forms distributed to early voters'),
 ('Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
  'Number of voting forms distributed at the polling station on the day of the election'),
 ('Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования',
  'Number of voting forms distributed outside the polling station on the day of the election'),
 ('Число погашенных избирательных бюллетеней',
  'Number of destroyed voting forms'),
 ('Число избирательных бюллетеней в переносных ящиках для голосования',
  'Number of voting forms in mobile boxes'),
 ('Число бюллетеней в стационарных ящиках для гол

In [12]:
# Rows correspond to a certain measurement, the columns correspond to the local electoral committees.
# The list of candidates is the same across the entire country.
print(results_table["row_headers"][12:])  # snipped for brevity

['Бабурин Сергей Николаевич', 'Грудинин Павел Николаевич', 'Жириновский Владимир Вольфович', 'Путин Владимир Владимирович', 'Собчак Ксения Анатольевна', 'Сурайкин Максим Александрович', 'Титов Борис Юрьевич', 'Явлинский Григорий Алексеевич']


In [13]:
#
# The *_uik tables contain the same row headers as their regular counterparts.
# They just show finer-grained information.
#
uik_table = [t for t in tables if t["data_type"] == "results_uik"][0]
assert uik_table["row_headers"] == results_table["row_headers"]
uik_table["column_headers"][:5]  # snipped for brevity

['Сумма', 'УИК №871', 'УИК №872', 'УИК №873', 'УИК №874']

In [14]:
# They also include the territory electoral committe (территориальная избирательная комиссия)
uik_table["territory"]

'Хоринская'

In [15]:
# The data itself can be obtained by indexing into the data matrix.
results_table["data"][0][0]

10382.0

# Checking the Dataset

In [16]:
# Make sure we have complete data for each
counter = collections.Counter(table["data_type"] for table in tables)
counter

Counter({'results_tik': 87,
         'results_uik': 2776,
         'turnout_tik': 87,
         'turnout_uik': 2776})

# Summary

In [17]:
# The number of OIK (region electoral committees), TIK (territory electoral committees),
# and UIK (spot electoral committees) per region
import csv
import sys

writer = csv.writer(sys.stdout, delimiter="|")
writer.writerow(["region", "oik", "tik", "uik"])

results = [t for t in tables if t["data_type"] == "results_tik"]
results_uik = [t for t in tables if t["data_type"] == "results_uik"]
oik_counter = collections.Counter(t["region"] for t in results)

for region, num_oik in sorted(oik_counter.items()):
    region_tables = [t for t in results if t["region"] == region]
    num_tik = sum([(len(t["column_headers"]) - 1) for t in region_tables])
    
    region_tables_uik = [t for t in results_uik if t["region"] == region]
    num_uik = sum([(len(t["column_headers"]) - 1) for t in region_tables_uik])
    
    writer.writerow([region, num_oik, num_tik, num_uik])

region|oik|tik|uik
Алтайский край|1|74|1831
Амурская область|1|29|772
Архангельская область|1|31|948
Астраханская область|1|16|579
Белгородская область|1|22|1263
Брянская область|1|34|1113
Владимирская область|1|23|984
Волгоградская область|1|46|1539
Вологодская область|1|28|1000
Воронежская область|1|39|1717
Город Байконур (Республика Казахстан)|1|0|0
Еврейская автономная область|1|6|184
Забайкальский край|1|38|942
Ивановская область|1|30|783
Иркутская область|1|45|1914
Кабардино-Балкарская Республика|1|13|354
Калининградская область|1|24|0
Калужская область|1|28|732
Камчатский край|1|14|317
Карачаево-Черкесская Республика|1|12|251
Кемеровская область|1|46|1750
Кировская область|1|48|1178
Костромская область|1|31|600
Краснодарский край|1|60|2791
Красноярский край|1|70|2202
Курганская область|1|27|994
Курская область|1|35|1163
Ленинградская область|1|18|985
Липецкая область|1|24|911
Магаданская область|1|10|104
Московская область|1|73|4238
Мурманская область|1|17|596
Ненецкий автономны

# Grouping results and turnout by polling station

This format of the data may be easier to work with for some applications.

In [18]:
def yield_turnout():
    turnout = [t for t in tables if t['data_type'] == 'turnout_uik']
    for table in turnout:
        for i, (header, row) in enumerate(zip(table['row_headers'], table['data'])):
            if i == 0:
                # skip total row
                continue
            yield {'region': table['region'], 'territory': table['territory'],
                   'polling_station': header, 'turnout': row}
                

turnout_dict = {}
for d in yield_turnout():
    key = (d['region'], d['territory'], d['polling_station'])
    turnout_dict[key] = d['turnout']

In [19]:
TIMES = ['10:00', '12:00', '15:00', '18:00']
list(turnout_dict.items())[0]

(('Республика Алтай', 'Чемальская', 'УИК №181'), [10.03, 28.44, 45.79, 58.82])

In [20]:
def yield_grouped():
    results = [t for t in tables if t['data_type'] == 'results_uik']
    for table in results:
        assert table['row_headers'] == ROW_HEADERS
        for column_number, column_name in enumerate(table['column_headers']):
            if column_number == 0:
                continue
            key = (table['region'], table['territory'], column_name)
            results = {dim: table['data'][row_number][column_number]
                       for (row_number, dim) in enumerate(ROW_HEADERS)}
            yield {
                'region': table['region'], 'territory': table['territory'],
                'polling_station': column_name, 'turnout': turnout_dict[key],
                'results': results
            }
            
grouped = list(yield_grouped())
grouped[1]

{'polling_station': 'УИК №872',
 'region': 'Республика Бурятия',
 'results': {'Бабурин Сергей Николаевич': 0.0,
  'Грудинин Павел Николаевич': 10.0,
  'Жириновский Владимир Вольфович': 1.0,
  'Путин Владимир Владимирович': 111.0,
  'Собчак Ксения Анатольевна': 7.0,
  'Сурайкин Максим Александрович': 0.0,
  'Титов Борис Юрьевич': 0.0,
  'Число бюллетеней в стационарных ящиках для голосования': 128.0,
  'Число действительных избирательных бюллетеней': 132.0,
  'Число избирателей, включенных в список избирателей': 133.0,
  'Число избирательных бюллетеней в переносных ящиках для голосования': 5.0,
  'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования': 128.0,
  'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования': 5.0,
  'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно': 0.0,
  'Число избирательных бюллетеней, не учтенных при получении': 0.0,
  'Число избирательных бюллетеней, получен